In [ ]:
import sys
import warnings
import time

from pyspark.sql import SparkSession
from pyspark.sql.types import (
    StructType,
    StructField,
    StringType,
    FloatType,
    IntegerType,
)
import pandas as pd

from model.model import Model
from axis.axis import Axis
from output.values_goals import ValuesGoals
from data_processing.flujo_down import DownBase
from data_processing.flujo_mid import FlujoMid
from data_processing.flujo_union import DataProcessing
from data_processing.flujo_caja_utils import union_flujos, union_flujos_2, get_vpn_calculado
from utils.databricks_hivemetastore import DatabricksHive
from utils.logger import Logger
from utils.general_utils import read_json, auto_cast_df, get_configuration, get_configuration_model, try_float, check_units, get_run_params, spark_cast, save_table_real, save_capex_debt, get_colombian_time, import_model_results, save_types_union, save_db_name_csv

warnings.simplefilter(action="ignore", category=Warning)

logger = Logger(__name__).get_logger()

In [ ]:
# Definición de funciones
def measure_time(task_name, func, *args):
    """
    Executes a function, measures its execution time, and logs the start, end, and elapsed time.

    :param task_name: Name or description of the task being measured.
    :type task_name: str
    :param func: The function that needs its execution time to be measured.
    :type func: function
    :param *args: Variable length argument list of the function's parameters.
    :type *args: tuple
    :return: Returns what the input function returns.
    :rtype: varies based on the function
    """
    logger.info(f"{task_name} successfully started")
    start_time = time.time()

    result = func(*args)

    end_time = time.time()
    elapsed_time = (end_time - start_time) / 60
    logger.info(f"{task_name} successfully finished, {elapsed_time:.2f} minutes")

    return result

def process_fc_up_be_isa_down(data_processing):
    """
    Processes the given data and resets its index.

    :param data_processing: An instance of a class with the fc_up_be_isa_down() method.
    :type data_processing: Object
    :return: DataFrame with reset index.
    :rtype: pd.DataFrame
    """
    df,df2 = data_processing.fc_up_be_isa_down()
    return df.reset_index(drop=True),df2

def run_model(model_config, params, df_union, df_down, df_mid, initial=False):
    """
    Runs a machine learning model using the provided configurations and data.

    :param model_config: Configuration parameters for the Model.
    :type model_config: dict
    :param params: Additional parameters for the model.
    :type params: dict
    :param df_union: DataFrame containing union data.
    :type df_union: pd.DataFrame
    :param df_down: DataFrame containing down base data.
    :type df_down: pd.DataFrame
    :param df_mid: DataFrame containing mid base data.
    :type df_mid: pd.DataFrame
    :param initial: bool to tell if this is a configuration run or a normal run
    :type initial: bool
    :return: Results and parameters of the run model.
    :rtype: tuple(pd.DataFrame, pd.DataFrame)
    """
    model = Model(model_config, params, df_union, df_down, df_mid, initial)
    return model.run_generations()

def get_down_base(df_model_results, config, api_config,input_up):
    """
    Generates a down base DataFrame based on model results and configuration.

    :param df_model_results: DataFrame containing results of a model run.
    :type df_model_results: pd.DataFrame
    :param config: Configuration parameters for DownBase.
    :type config: dict
    :param api_config: Configuration parameters from powerBI
    :type api_config: dict
    :return: DataFrame representing the down base with set column values.
    :rtype: pd.DataFrame
    """
    base = DownBase(df_model_results, config, api_config,input_up)
    df = base.get_base(False)
    df.loc[:, "TIPO_FLUJO_CAJA_FLAG"] = "REAL"
    return df

def get_mid_base(df_model_results, config, api_config,model_results):
    """
    Generates a mid base DataFrame based on model results and configuration.

    :param df_model_results: DataFrame containing results of a model run.
    :type df_model_results: pd.DataFrame
    :param config: Configuration parameters for MidBase.
    :type config: dict
    :param api_config: Configuration parameters from powerBI
    :type api_config: dict
    :return: DataFrame representing the mid base with set column values.
    :rtype: pd.DataFrame
    """
    base = FlujoMid(df_model_results, config, api_config,model_results)
    df = base.calculate_mid()
    df.loc[:, "TIPO_FLUJO_CAJA_FLAG"] = "REAL"
    return df

def get_down_base(df_model_results, config, api_config):
    """
    Generates a down base DataFrame based on model results and configuration.

    :param df_model_results: DataFrame containing results of a model run.
    :type df_model_results: pd.DataFrame
    :param config: Configuration parameters for DownBase.
    :type config: dict
    :param api_config: Configuration parameters from powerBI
    :type api_config: dict
    :return: DataFrame representing the down base with set column values.
    :rtype: pd.DataFrame
    """
    base = DownBase(df_model_results, config, api_config)
    df = base.get_base(False)
    df.loc[:, "TIPO_FLUJO_CAJA_FLAG"] = "REAL"
    return df

In [ ]:
main_config = read_json("config/main_config.json")
data_source_config = read_json("config/data_source_config.json")
model_config = get_configuration_model(main_config["config_path"], "config/model_config.json")
total_caja_config = get_configuration(main_config["config_path"], "config/fuente_config.json")

In [ ]:
params = [0,30,20,2.5,40,3.17,40,13,0,30,0,6.75,0,647.5,0,2001,0,230,0,2,0,2,0,2,0,
3000,0,21,1,100,100,100,100,100,50,50,0.5,0.5,1,1,1,0,0,0,"momento1"]
params_encabezado = [
"vpn_weight",
"vpn_goal",
"deuda_bruta_weight",
"deuda_bruta_ratio",
"trans_nacion_weight",
"trans_nacion_min",
"ebitda_weight",
"ebitda_min",
"low_emissions_weight",
"low_emissions_min",
"emisiones_netas_co2_alcance_1_y_2_weight",
"emisiones_netas_co2_alcance_1_y_2_actual",
"neutralidad_agua_weight",
"neutralidad_agua_actual",
"mwh_weight",
"mwh_goal",
"not_oil_jobs_weight",
"not_oil_jobs_goal",
"students_weight",
"students_goal",
"natural_gas_weight",
"natural_gas_actual",
"agua_potable_weight",
"agua_potable_actual",
"km_weight",
"km_actual",
"cti_weight",
"cti_min",
"val_curva_crudo",
"val_sensi_precio_crudo",
"val_sensi_precio_gas",
"val_sensi_precio_Hidro",
"val_delta_energia",
"val_sensi_precio_co2",
"generations",
"sol_per_pop",
"mutation_ind",
"mutation_gen",
"vpn_zero_restriction",
"be_restriction",
"phase_restriction",
"cash_flow_restriction",
"gas_demanda_nacion",
"transporte_offshore",
"momento"]



In [ ]:
params_dic = dict(zip(params_encabezado, params))
params_to_float = params_dic

In [ ]:
# get_inital_base
dfs_dict_result = {}
data_processing = DataProcessing(total_caja_config, params_to_float)


In [ ]:
flujo_union, df_flujo_caja_up = measure_time(
    "fc_up_be_isa_down", process_fc_up_be_isa_down, data_processing
)

flujo_union = get_vpn_calculado(flujo_union)

In [ ]:
flujo_union["MATRICULA_DIGITAL"] = flujo_union["MATRICULA_DIGITAL"].astype(str)
flujo_union["DEPENDENCIAS"] = flujo_union["DEPENDENCIAS"].astype(str)
flujo_union["EXCLUYENTES"] = flujo_union["EXCLUYENTES"].astype(str)

In [ ]:
(
    model_results,
    model_fitness
) = measure_time(
    "Model",
    run_model,
    model_config,
    params_to_float,
    flujo_union,
    None,
    None,
    True
)

In [ ]:
model_results.to_csv("output/model_results_240424.csv", index=False,sep = ";")

In [ ]:
model.df_union.to_csv("output/df_union_240424.csv", index=False,sep = ";")

In [ ]:
flujo_down_real = measure_time(
    "DownBase",
    get_down_base,
    model_results,
    total_caja_config,
    params_to_float
)

In [ ]:
model = Model(model_config, #config_dict
    params_to_float, #pb_parameters
    flujo_union, #input_data
    None, #down_data
    None, #mid_data
    True) #initial

In [ ]:
model.df_projects.to_csv("output/df_projects_240424.csv", index=False,sep = ";")
model.df_vpn_zero_projects.to_csv("output/df_vpn_zero_projects_240424.csv", index=False,sep = ";")
model.df_mandatory_projects.to_csv("output/df_mandatory_projects_240424.csv", index=False,sep = ";")
model.df_semi_mandatory.to_csv("output/df_semi_mandatory_240424.csv", index=False,sep = ";")


In [ ]:
model.df_projects["MATRICULA_DIGITAL"] = model.df_projects["MATRICULA_DIGITAL"].astype(str)
model.df_projects["DEPENDENCIAS"] = model.df_projects["DEPENDENCIAS"].astype(str)
model.df_projects["EXCLUYENTES"] = model.df_projects["EXCLUYENTES"].astype(str)

In [ ]:
model.df_mandatory_projects["MATRICULA_DIGITAL"].unique().tolist()+model.df_semi_mandatory["MATRICULA_DIGITAL"].unique().tolist()+model.df_vpn_zero_projects["MATRICULA_DIGITAL"].unique().tolist()

In [ ]:
pyga_instance = model.init_pygad()



In [ ]:
pyga_instance.run()

In [ ]:
def select_projects(df):
    """
    Selects projects based on their dependencies and exclusions.

    This function takes a DataFrame containing project information as input and selects projects that can be executed considering their dependencies and exclusions.

    :param df: DataFrame containing project information including 'Proyecto' (Project), 'Dependencias' (Dependencies), and 'Excluyentes' (Exclusions).
    :type df: pandas.DataFrame

    :return: List of projects that are available for selection based on their dependencies and exclusions.
    :rtype: list
    """
    available_projects = []
    selected_projects = set()
    exclusive_projects = set()
    for index, row in df.iterrows():
        project = row['MATRICULA_DIGITAL']
        dependencies = [dep.strip() for dep in row['DEPENDENCIAS'].split(',')]
        if dependencies == ['NA']:
            dependencies = []
        exclusions = [ex.strip() for ex in row['EXCLUYENTES'].split(',')]
        if exclusions == ['NA']:
            exclusions = []

        # Check if all dependencies are satisfied and the project is not excluded
        if len(dependencies)+len(exclusions) >0:
            if all(dep in selected_projects for dep in dependencies) and not any(ex in exclusive_projects for ex in exclusions):
                available_projects.append(project)
        else:
            available_projects.append(project)

        # Add the project to selected projects and update exclusions
        selected_projects.add(project)
        exclusive_projects.update(exclusions)
    return available_projects

In [ ]:
model.df_projects["DEPENDENCIAS"]

In [ ]:
import random

def select_projects_random(df):
    available_projects = []
    selected_projects = set()
    exclusive_projects = set()
    
    
    for index, row in df.iterrows():
        project = row['MATRICULA_DIGITAL']
        dependencies = [dep.strip() for dep in row['DEPENDENCIAS'].split(',')]
        if dependencies == ['NA']:
            dependencies = []
        exclusions = [ex.strip() for ex in row['EXCLUYENTES'].split(',')]
        if exclusions == ['NA']:
            exclusions = []
       
        # Verificar si las dependencias están satisfechas.
        if all(dep in selected_projects for dep in dependencies):
            # Verificar si el proyecto no es excluido por cualquier otro proyecto.
            excluded = False
            for ex in exclusions:
                if ex in selected_projects:
                    excluded = True
                    break
            # Si no está excluido, añadir el proyecto a la lista de proyectos disponibles.
            if not excluded:
                available_projects.append(project)
     
        # Añadir el proyecto a los proyectos seleccionados y actualizar las exclusiones.
        selected_projects.add(project)
        exclusive_projects.update(exclusions)
      
        # Aleatorizar el orden para las exclusiones para la siguiente iteración.
        exclusive_projects_list = list(exclusive_projects)
        random.shuffle(exclusive_projects_list)
        exclusive_projects = set(exclusive_projects_list)
   
    return available_projects


In [ ]:
import pandas as pd


In [ ]:
data = {
    'MATRICULA_DIGITAL': ['Proyecto B ETAPA 2', 'Proyecto A ETAPA 2', 'Proyecto A ETAPA 1', 'Proyecto C', 'Proyecto B ETAPA 1','Proyecto X','Proyecto Y','Proyecto Z','Proyecto AA','Proyecto AB'],
    'DEPENDENCIAS': ['Proyecto B ETAPA 1, Proyecto C', 'Proyecto A ETAPA 1', '', '', 'Proyecto A ETAPA 2','','','','',''],
    'EXCLUYENTES': ['', '', '', 'Proyecto A ETAPA 1, Proyecto A ETAPA 2', '','','','Proyecto AA, Proyecto AB','Proyecto Z, Proyecto AB','Proyecto Z, Proyecto AA'],
}
 
dataframe = pd.DataFrame(data)

In [ ]:
dataframe

In [ ]:
mutaciones = [1,1,1,1,1,1,0,1,1,0]
mutaciones_filtro = [True if mut == 1 else False for mut in mutaciones]
mutaciones_filtro

In [ ]:
not_available = dataframe["MATRICULA_DIGITAL"][[False if mut == 1 else True for mut in mutaciones]].tolist()
not_available

In [ ]:
dataframe2 = dataframe[mutaciones_filtro]
dataframe2

In [ ]:
# verificar proyectos con exclusiones 

available_projects = dataframe2["MATRICULA_DIGITAL"][(dataframe2["EXCLUYENTES"]=="")].tolist()
lista_pro_exl = [proyecto for proyecto in dataframe2["MATRICULA_DIGITAL"].unique().tolist() if proyecto not in available_projects]
excluded_projects = []
#excluded_projects = dataframe["MATRICULA_DIGITAL"][[False if mut == 1 else True for mut in mutaciones]]

In [ ]:
i= 0
n = len(lista_pro_exl)
while len(lista_pro_exl) > 0:
    i+=1
    print(i)
    if i>n:
        break
    df_trabajo = dataframe2[dataframe2["MATRICULA_DIGITAL"].isin(lista_pro_exl)]
    for index, row in df_trabajo.iterrows():
        project = row['MATRICULA_DIGITAL']
        print("proyecto que se esta evaluando "+ str(project))
        exclusions = [ex.strip() for ex in row['EXCLUYENTES'].split(',')]
        
        #Verificar si todas las exclusiones están en los proyectos seleccionados
        es_aleatorio = False
        lista_aleatorio = [project]        
        for ex in exclusions:
            if ex in lista_pro_exl_ori:
                exclusiones_ex = [ex.strip() for ex in df_trabajo["EXCLUYENTES"][df_trabajo["MATRICULA_DIGITAL"]==ex].values[0].split(',')]
                if project in exclusiones_ex:
                    es_aleatorio = True
                    lista_aleatorio.append(ex)
        print(lista_aleatorio)
        print("Es aleatorio "+str(es_aleatorio))  
        if es_aleatorio:
            pro_sel_ale = random.choice(lista_aleatorio)
            print("Proyecto seleccionado aleatoriamente "+str(pro_sel_ale))
            lista_pro_exl.remove(pro_sel_ale)
            lista_aleatorio.remove(pro_sel_ale)
            available_projects.append(pro_sel_ale)
            excluded_projects = excluded_projects+lista_aleatorio
            print("Proyectos excluidos "+str(lista_aleatorio))
            for pro in lista_aleatorio:
                lista_pro_exl.remove(pro)
            break
        else:
            if all (ex in available_projects + excluded_projects for ex in exclusions):
                # Verificar si el proyecto no es excluido por cualquier otro proyecto.
                excluded = False
                for ex in exclusions:
                    if ex in available_projects:
                        excluded = True
                        break
                lista_pro_exl.remove(project)
                if excluded:
                    excluded_projects.append(project)
                else:
                    selected_projects.append(project)
            else:
                pass


In [ ]:
available_projects

In [ ]:
excluded_projects

In [ ]:
dataframe3 = dataframe2[dataframe2["MATRICULA_DIGITAL"].isin(available_projects)]
dataframe3

In [ ]:
select_projects = dataframe3[dataframe3["DEPENDENCIAS"] == ""]["MATRICULA_DIGITAL"].tolist()
lista_pro_dep = [proyecto for proyecto in dataframe3["MATRICULA_DIGITAL"].unique().tolist() if proyecto not in select_projects]

In [ ]:

i = 0
n = len(lista_pro_dep)
while len(lista_pro_dep) > 0:
    i+=1
    if i>n:
        break
    df_trabajo = dataframe3[dataframe3["MATRICULA_DIGITAL"].isin(lista_pro_dep)]
    for index, row in df_trabajo.iterrows():
        project = row['MATRICULA_DIGITAL']
        dependencies = [dep.strip() for dep in row['DEPENDENCIAS'].split(',')]
        if dependencies == ['NA']:
            dependencies = []
        # Verificar si las dependencias están satisfechas.
        if all(dep in select_projects + excluded_projects for dep in dependencies):
            if all(dep in select_projects for dep in dependencies):
                select_projects.append(project)
                lista_pro_dep.remove(project)
            else:
                if any(dep in excluded_projects for dep in dependencies):
                    excluded_projects.append(project)
                    lista_pro_dep.remove(project)
        else:
            pass


In [ ]:
dataframe2

In [ ]:
select_projects

In [ ]:
excluded_projects

## Encapsulación del bucle como método

Se puede encapsular el bucle de selección de proyectos en una función para hacer el código más modular y reutilizable. Acá está el código modificado para que la selección de proyectos excluidos esté dentro de una función llamada seleccionar_proyectos_excluidos.

In [4]:
import pandas as pd
import random

def seleccionar_proyectos_excluidos(mutants, df_projects):
    """
    Selects projects considering their dependencies and exclusions.

    This function takes a DataFrame containing project information as input and selects projects that can be executed considering 
    their dependencies and exclusions.
    
    :param mutant: np.array containing the portafolio to be checked
    :type mutant: np.array
    :param df_projects: df containing projects
    :type df_projects: DataFrame

    :return: Tuple containing lists of projects that are available and excluded for selection based on their dependencies and exclusions.
    :rtype: tuple
    """
    available_projects = []
    excluded_projects = []
    lista_pro_exl_ori = df_projects["MATRICULA_DIGITAL"].unique().tolist()
    lista_pro_exl = lista_pro_exl_ori.copy()

    for i in range(len(mutants)):
        if mutants[i] == 1:
            project = df_projects.loc[i, 'MATRICULA_DIGITAL']
            exclusions = [ex.strip() for ex in df_projects.loc[i, 'EXCLUYENTES'].split(',')]

            if all(ex in available_projects + excluded_projects for ex in exclusions):
                excluded = False
                for ex in exclusions:
                    if ex in available_projects:
                        excluded = True
                        break
                lista_pro_exl.remove(project)
                if excluded:
                    excluded_projects.append(project)
                else:
                    available_projects.append(project)
            else:
                pass

    i = 0
    n = len(lista_pro_exl)
    while len(lista_pro_exl) > 0:
        i += 1
        if i > n:
            break
        df_trabajo = df_projects[df_projects["MATRICULA_DIGITAL"].isin(lista_pro_exl)]
        for index, row in df_trabajo.iterrows():
            project = row['MATRICULA_DIGITAL']
            exclusions = [ex.strip() for ex in row['EXCLUYENTES'].split(',')]
            
            es_aleatorio = False
            lista_aleatorio = [project]
            for ex in exclusions:
                if ex in lista_pro_exl_ori:
                    exclusiones_ex = [ex.strip() for ex in df_trabajo["EXCLUYENTES"][df_trabajo["MATRICULA_DIGITAL"]==ex].values[0].split(',')]
                    if project in exclusiones_ex:
                        es_aleatorio = True
                        lista_aleatorio.append(ex)
            
            if es_aleatorio:
                pro_sel_ale = random.choice(lista_aleatorio)
                lista_pro_exl.remove(pro_sel_ale)
                lista_aleatorio.remove(pro_sel_ale)
                available_projects.append(pro_sel_ale)
                excluded_projects.extend(lista_aleatorio)
                for pro in lista_aleatorio:
                    lista_pro_exl.remove(pro)
                break
            else:
                if all(ex in available_projects + excluded_projects for ex in exclusions):
                    excluded = False
                    for ex in exclusions:
                        if ex in available_projects:
                            excluded = True
                            break
                    lista_pro_exl.remove(project)
                    if excluded:
                        excluded_projects.append(project)
                    else:
                        available_projects.append(project)
                else:
                    pass

    return available_projects, excluded_projects

# Datos de ejemplo
data = {
    'MATRICULA_DIGITAL': ['Proyecto B ETAPA 2', 'Proyecto A ETAPA 2', 'Proyecto A ETAPA 1', 'Proyecto C', 'Proyecto B ETAPA 1','Proyecto X','Proyecto Y','Proyecto Z','Proyecto AA','Proyecto AB'],
    'DEPENDENCIAS': ['Proyecto B ETAPA 1, Proyecto C', 'Proyecto A ETAPA 1', '', '', 'Proyecto A ETAPA 2','','','','',''],
    'EXCLUYENTES': ['', '', '', 'Proyecto A ETAPA 1, Proyecto A ETAPA 2', '','','','Proyecto AA, Proyecto AB','Proyecto Z, Proyecto AB','Proyecto Z, Proyecto AA'],
}

df_projects = pd.DataFrame(data)
mutants = [1,1,1,1,1,1,0,1,1,0]

available_projects, excluded_projects = seleccionar_proyectos_excluidos(mutants, df_projects)
print("Proyectos disponibles:", available_projects)
print("Proyectos excluidos:", excluded_projects)



Proyectos disponibles: ['Proyecto Z']
Proyectos excluidos: ['Proyecto AA', 'Proyecto AB']
